In [1]:
# Eval code, adapted for jupyter notebook with backtest decorator
import numpy as np
import pandas as pd
from functools import wraps

nInst = 0
nt = 0
commRate = 0.0005
dlrPosLimit = 10000

def loadPrices(fn):
    global nt, nInst
    df=pd.read_csv(fn, sep=r'\s+', header=None, index_col=None)
    (nt,nInst) = df.shape
    return (df.values).T

pricesFile="./.data/2025/prices.txt"
prcAll = loadPrices(pricesFile)
print ("Loaded %d instruments for %d days" % (nInst, nt))

def calcPL(getPosition, prcHist=None, numTestDays=200, stock_list=None, show_daily_positions=False):
    if prcHist is None:
        prcHist = prcAll
    
    # Determine which stocks to evaluate
    if stock_list is not None:
        eval_stocks = stock_list
        num_eval_stocks = len(eval_stocks)
    else:
        eval_stocks = list(range(nInst))
        num_eval_stocks = nInst
    
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    
    # Track per-stock performance for evaluated stocks only
    stock_pnl_history = []  # Daily P&L per evaluated stock
    stock_total_pnl = np.zeros(num_eval_stocks)  # Cumulative P&L per evaluated stock
    
    (_,nt_local) = prcHist.shape
    startDay = nt_local + 1 - numTestDays
    
    for t in range(startDay, nt_local+1):
        prcHistSoFar = prcHist[:,:t]
        curPrices = prcHistSoFar[:,-1]
        
        if (t < nt_local):
            # Trading, do not do it on the very last day of the test
            newPosOrig = getPosition(prcHistSoFar)
            posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
            newPos = np.clip(newPosOrig, -posLimits, posLimits)
            deltaPos = newPos - curPos
            dvolumes = curPrices * np.abs(deltaPos)
            dvolume = np.sum(dvolumes)
            totDVolume += dvolume
            comm = dvolume * commRate
            cash -= curPrices.dot(deltaPos) + comm
        else:
            newPos = np.array(curPos)
        
        # Calculate per-stock P&L for this day (only for evaluated stocks)
        if t > startDay:
            prevPrices = prcHist[:, t-2] if t > 1 else prcHistSoFar[:, -2]
            
            # Calculate P&L for all stocks
            all_stock_daily_pnl = curPos * (curPrices - prevPrices)
            
            # Extract P&L only for evaluated stocks
            eval_stock_daily_pnl = np.array([all_stock_daily_pnl[i] for i in eval_stocks])
            stock_total_pnl += eval_stock_daily_pnl
            stock_pnl_history.append(eval_stock_daily_pnl.copy())
        
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        value = cash + posValue
        
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        if (t > startDay):
            print ("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" % (t,value, todayPL, totDVolume, ret))
            
            # Show daily positions if requested
            if show_daily_positions:
                print(f"  Cash: ${cash:.2f} | Position Value: ${posValue:.2f}")
                
                # Show positions for evaluated stocks only
                active_positions = []
                for i, stock_idx in enumerate(eval_stocks):
                    if abs(curPos[stock_idx]) > 0:
                        pos_value = curPos[stock_idx] * curPrices[stock_idx]
                        active_positions.append(f"S{stock_idx}:{int(curPos[stock_idx])}@{curPrices[stock_idx]:.2f}=${pos_value:.0f}")
                
                if active_positions:
                    print(f"  Active Positions: {' | '.join(active_positions)}")
                else:
                    print("  Active Positions: None")
                
                # Show top position changes if any
                if t > startDay + 1:
                    prev_pos = curPos - deltaPos
                    position_changes = []
                    for i, stock_idx in enumerate(eval_stocks):
                        if abs(deltaPos[stock_idx]) > 0:
                            change_value = deltaPos[stock_idx] * curPrices[stock_idx]
                            position_changes.append(f"S{stock_idx}:{int(deltaPos[stock_idx]):+}=${change_value:+.0f}")
                    
                    if position_changes:
                        print(f"  Position Changes: {' | '.join(position_changes)}")
                
                print()  # Extra line for readability
            
            todayPLL.append(todayPL)
    
    pll = np.array(todayPLL)
    (plmu,plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(249) * plmu / plstd
    
    # Calculate per-stock Sharpe ratios for evaluated stocks only
    if len(stock_pnl_history) > 0:
        stock_pnl_array = np.array(stock_pnl_history)  # Shape: (num_days, num_eval_stocks)
        stock_sharpe_ratios = np.zeros(num_eval_stocks)
        
        for i in range(num_eval_stocks):
            stock_daily_pnl = stock_pnl_array[:, i]
            stock_mean_pnl = np.mean(stock_daily_pnl)
            stock_std_pnl = np.std(stock_daily_pnl)
            
            if stock_std_pnl > 0:
                stock_sharpe_ratios[i] = np.sqrt(249) * stock_mean_pnl / stock_std_pnl
            else:
                stock_sharpe_ratios[i] = 0.0
    else:
        stock_sharpe_ratios = np.zeros(num_eval_stocks)
    
    return (plmu, ret, plstd, annSharpe, totDVolume, stock_total_pnl, stock_sharpe_ratios)

def backtest(numTestDays=200, prcHist=None, verbose=True, stock_list=None, show_daily_positions=False):
    """
    Decorator to backtest a position function
    
    Args:
        numTestDays: Number of days to test (default 200)
        prcHist: Price history array (default uses global prcAll)
        verbose: Whether to print detailed results (default True)
        stock_list: List of stock indices to evaluate (default None = all stocks)
        show_daily_positions: Show daily position details (default False)
    """
    def decorator(getPosition_func):
        @wraps(getPosition_func)
        def wrapper(*args, **kwargs):
            print(f"Testing function: {getPosition_func.__name__}")
            
            # Determine which stocks to evaluate
            if stock_list is not None:
                eval_stocks = stock_list
                print(f"Evaluating subset: {len(eval_stocks)} stocks {eval_stocks}")
            else:
                eval_stocks = list(range(nInst))
                print(f"Evaluating all {nInst} stocks")
            
            # Run the backtest
            result = calcPL(
                getPosition_func, 
                prcHist if prcHist is not None else prcAll, 
                numTestDays,
                stock_list=eval_stocks,
                show_daily_positions=show_daily_positions
            )
            
            if len(result) == 7:  # New format with per-stock data
                (meanpl, ret, plstd, sharpe, dvol, stock_pnl, stock_sharpe) = result
            else:  # Old format for backward compatibility
                (meanpl, ret, plstd, sharpe, dvol) = result
                stock_pnl = np.zeros(len(eval_stocks))
                stock_sharpe = np.zeros(len(eval_stocks))
            
            score = meanpl - 0.1*plstd
            
            if verbose:
                print ("=====")
                print (f"Function: {getPosition_func.__name__}")
                print ("mean(PL): %.1lf" % meanpl)
                print ("return: %.5lf" % ret)
                print ("StdDev(PL): %.2lf" % plstd)
                print ("annSharpe(PL): %.2lf " % sharpe)
                print ("totDvolume: %.0lf " % dvol)
                print ("Score: %.2lf" % score)
                print ("=====")
                
                # Print per-stock performance
                print("\nPER-STOCK PERFORMANCE:")
                print("Stock#\tTotal P&L\tAnn Sharpe")
                print("-" * 35)
                for i, stock_idx in enumerate(eval_stocks):
                    print(f"{stock_idx:2d}\t{stock_pnl[i]:8.1f}\t{stock_sharpe[i]:8.2f}")
                
                # Summary statistics for stocks
                profitable_stocks = np.sum(stock_pnl > 0)
                losing_stocks = np.sum(stock_pnl < 0)
                best_stock_pnl = np.max(stock_pnl)
                worst_stock_pnl = np.min(stock_pnl)
                best_stock_sharpe = np.max(stock_sharpe)
                worst_stock_sharpe = np.min(stock_sharpe)
                
                print("-" * 35)
                print(f"Profitable stocks: {profitable_stocks}/{len(stock_pnl)}")
                print(f"Best stock P&L: {best_stock_pnl:.1f}")
                print(f"Worst stock P&L: {worst_stock_pnl:.1f}")
                print(f"Best stock Sharpe: {best_stock_sharpe:.2f}")
                print(f"Worst stock Sharpe: {worst_stock_sharpe:.2f}")
                print(f"Avg stock Sharpe: {np.mean(stock_sharpe):.2f}")
                print("=====")
            
            # Store results as function attributes for later access
            wrapper.results = {
                'meanpl': meanpl,
                'return': ret,
                'plstd': plstd,
                'sharpe': sharpe,
                'dvol': dvol,
                'score': score,
                'stock_pnl': stock_pnl,
                'stock_sharpe': stock_sharpe,
                'eval_stocks': eval_stocks
            }
            
            return wrapper.results
        
        # Run backtest immediately when decorator is applied
        wrapper()
        return wrapper
    
    return decorator



Loaded 50 instruments for 747 days


In [21]:
@backtest()
def simple_long_position(prcHistSoFar):
    """Simple strategy: starts with 1 share and increases by 1 each day"""
    nInst, nt = prcHistSoFar.shape
    
    # Position size increases with time
    # Day 1: 1 share each, Day 2: 2 shares each, etc.
    position_size = nt - 1  # Subtract 1 because we start from day 1
    
    return np.full(nInst, max(1, position_size))

Testing function: simple_long_position
Evaluating all 50 stocks
Day 549 value: -949.92 todayPL: $-702.72 $-traded: 498257 return: -0.00191
Day 550 value: -1408.35 todayPL: $-458.43 $-traded: 501793 return: -0.00281
Day 551 value: -1795.91 todayPL: $-387.56 $-traded: 505393 return: -0.00355
Day 552 value: -1309.00 todayPL: $486.91 $-traded: 508394 return: -0.00257
Day 553 value: -1589.05 todayPL: $-280.05 $-traded: 511792 return: -0.00310
Day 554 value: -3466.08 todayPL: $-1877.03 $-traded: 515951 return: -0.00672
Day 555 value: -4230.40 todayPL: $-764.32 $-traded: 519339 return: -0.00815
Day 556 value: -3947.08 todayPL: $283.32 $-traded: 522831 return: -0.00755
Day 557 value: -5714.17 todayPL: $-1767.09 $-traded: 527050 return: -0.01084
Day 558 value: -6568.50 todayPL: $-854.34 $-traded: 530408 return: -0.01238
Day 559 value: -7087.47 todayPL: $-518.97 $-traded: 533863 return: -0.01328
Day 560 value: -7049.10 todayPL: $38.37 $-traded: 537316 return: -0.01312
Day 561 value: -7665.67 tod

In [3]:
@backtest()
def momentum_strategy(prcHistSoFar):
    """Simple momentum strategy"""
    nInst, nt = prcHistSoFar.shape
    if nt < 2:
        return np.zeros(nInst)
    
    # Calculate recent returns
    recent_returns = (prcHistSoFar[:, -1] - prcHistSoFar[:, -2]) / prcHistSoFar[:, -2]
    
    # Go long on positive momentum, short on negative|
    positions = np.sign(recent_returns)
    return positions


Testing function: momentum_strategy
Day 549 value: -3.21 todayPL: $-2.01 $-traded: 5022 return: -0.00064
Day 550 value: -3.51 todayPL: $-0.30 $-traded: 7723 return: -0.00045
Day 551 value: -4.00 todayPL: $-0.49 $-traded: 10098 return: -0.00040
Day 552 value: -4.10 todayPL: $-0.10 $-traded: 12560 return: -0.00033
Day 553 value: -7.69 todayPL: $-3.60 $-traded: 14390 return: -0.00053
Day 554 value: -10.48 todayPL: $-2.79 $-traded: 16850 return: -0.00062
Day 555 value: -10.78 todayPL: $-0.29 $-traded: 19371 return: -0.00056
Day 556 value: -13.39 todayPL: $-2.62 $-traded: 21868 return: -0.00061
Day 557 value: -19.82 todayPL: $-6.43 $-traded: 24522 return: -0.00081
Day 558 value: -20.83 todayPL: $-1.01 $-traded: 26974 return: -0.00077
Day 559 value: -28.09 todayPL: $-7.26 $-traded: 29954 return: -0.00094
Day 560 value: -26.42 todayPL: $1.67 $-traded: 32203 return: -0.00082
Day 561 value: -31.06 todayPL: $-4.64 $-traded: 35008 return: -0.00089
Day 562 value: -31.48 todayPL: $-0.42 $-traded: 3

In [7]:
@backtest()
def mean_reversion_strategy(prcHistSoFar):
    """Simple mean reversion strategy"""
    nInst, nt = prcHistSoFar.shape
    if nt < 10:
        return np.zeros(nInst)
    
    # Calculate deviation from 10-day mean
    recent_mean = np.mean(prcHistSoFar[:, -10:], axis=1)
    current_price = prcHistSoFar[:, -1]
    deviation = (current_price - recent_mean) / recent_mean
    
    # Go opposite to deviation (mean reversion)
    positions = -np.sign(deviation)
    return positions

Testing function: mean_reversion_strategy
Day 552 value: 1.18 todayPL: $2.41 $-traded: 3522 return: 0.00033
Day 553 value: -1.76 todayPL: $-2.94 $-traded: 3899 return: -0.00045
Day 554 value: 0.52 todayPL: $2.28 $-traded: 4934 return: 0.00011
Day 555 value: -0.34 todayPL: $-0.86 $-traded: 5520 return: -0.00006
Day 556 value: 1.08 todayPL: $1.42 $-traded: 6741 return: 0.00016
Day 557 value: 4.61 todayPL: $3.53 $-traded: 8202 return: 0.00056
Day 558 value: 3.92 todayPL: $-0.68 $-traded: 9031 return: 0.00043
Day 559 value: 7.05 todayPL: $3.12 $-traded: 10290 return: 0.00068
Day 560 value: 9.94 todayPL: $2.90 $-traded: 11572 return: 0.00086
Day 561 value: 6.74 todayPL: $-3.20 $-traded: 12342 return: 0.00055
Day 562 value: 7.17 todayPL: $0.44 $-traded: 13291 return: 0.00054
Day 563 value: 9.84 todayPL: $2.66 $-traded: 14502 return: 0.00068
Day 564 value: 13.21 todayPL: $3.37 $-traded: 16032 return: 0.00082
Day 565 value: 11.50 todayPL: $-1.72 $-traded: 17288 return: 0.00066
Day 566 value: 9

In [8]:
@backtest(250)
def trend_continuation_strategy(prcHistSoFar):
    """
    If trend continues for n consecutive days, position moves by n steps
    - Count consecutive up/down days for each stock
    - Position = number of consecutive days in same direction
    - Positive for uptrend, negative for downtrend
    """
    nInst, nt = prcHistSoFar.shape
    
    if nt < 3:  # Need at least 3 days to determine trend
        return np.zeros(nInst)
    
    positions = np.zeros(nInst)
    
    for i in range(nInst):
        prices = prcHistSoFar[i, :]
        consecutive_days = 0
        current_trend = 0  # 1 for up, -1 for down, 0 for flat
        
        # Look at daily changes
        for day in range(1, nt):
            daily_change = prices[day] - prices[day-1]
            
            if daily_change > 0:  # Up day
                if current_trend == 1:  # Continuing uptrend
                    consecutive_days += 1
                else:  # New uptrend starts
                    current_trend = 1
                    consecutive_days = 1
                    
            elif daily_change < 0:  # Down day
                if current_trend == -1:  # Continuing downtrend
                    consecutive_days += 1
                else:  # New downtrend starts
                    current_trend = -1
                    consecutive_days = 1
                    
            else:  # Flat day - reset
                current_trend = 0
                consecutive_days = 0
        
        # Position = consecutive_days * trend_direction
        positions[i] = consecutive_days * current_trend
    
    return positions

Testing function: trend_continuation_strategy
Day 499 value: 0.38 todayPL: $2.49 $-traded: 8506 return: 0.00004
Day 500 value: 11.46 todayPL: $11.08 $-traded: 12525 return: 0.00091
Day 501 value: 19.99 todayPL: $8.53 $-traded: 17364 return: 0.00115
Day 502 value: 11.79 todayPL: $-8.20 $-traded: 22867 return: 0.00052
Day 503 value: 10.58 todayPL: $-1.20 $-traded: 27715 return: 0.00038
Day 504 value: 19.76 todayPL: $9.18 $-traded: 32241 return: 0.00061
Day 505 value: 35.48 todayPL: $15.72 $-traded: 36354 return: 0.00098
Day 506 value: 63.85 todayPL: $28.37 $-traded: 40263 return: 0.00159
Day 507 value: 42.32 todayPL: $-21.53 $-traded: 47423 return: 0.00089
Day 508 value: 53.42 todayPL: $11.10 $-traded: 51877 return: 0.00103
Day 509 value: 33.48 todayPL: $-19.94 $-traded: 57573 return: 0.00058
Day 510 value: 35.51 todayPL: $2.03 $-traded: 61882 return: 0.00057
Day 511 value: 28.43 todayPL: $-7.08 $-traded: 67255 return: 0.00042
Day 512 value: 30.46 todayPL: $2.03 $-traded: 71890 return: 0

In [9]:

import numpy as np

##### TODO #########################################
### IMPLEMENT 'getMyPosition' FUNCTION #############
### TO RUN, RUN 'eval.py' ##########################

nInst = 50
currentPos = np.zeros(nInst, int)
openPositions = [[] for x in range(nInst)] #per instrument a list of open positions like [[pos, openPrice, lBound, uBound, type], [...]] bound==0 means bound inactive
#type == 0 is to check the bounds, type == 1 is to close pos on trend change or trend 0
firstCall = False
DESIREDRET = 3

def halfMeans(npa): #npa - multi dimensional array, return [means1, means2]
	(nins, nt) = npa.shape
	if nt <= 1:
		return ([npa, npa], [np.zeros(nins), np.zeros(nins)])
	if nt % 2 == 0:
		nt1 = nt // 2
		nt2 = nt1
#		halfa = (npa[:,nt1]+npa[:,np2])/2
	else:
		nt1 = nt // 2 + 1
		nt2 = nt1 - 1
#		halfa = npa[:,nt1]
	means = []
	std = []
#	print(nt1, nt2)
	means.append(npa[:,:nt1].mean(1))
	means.append(npa[:,nt2:].mean(1))
	std.append(npa[:,:nt1].std(1, ddof=1))
	std.append(npa[:,nt2:].std(1, ddof=1))
	return (means, std)

#daysPerPP is how many days do the calculation over
#returns py list with PPs list per instrument row; one pps is [high, low, mean of H,L,closingprice]
def calcPivotPoints(npa, daysPerPP): 
	(rows, cols) = npa.shape
	if daysPerPP < 2 or rows == 0 or cols == 0:
		return [[[e,e,e] for e in r] for r in npa]
	ret = []
	row = 0
	while row < rows:
		retrow = []
		col = cols
		while col > 0:
#			col -= 1
#			if d == 0:
#				d = daysPerPP
			lowerBound = col - daysPerPP
			if lowerBound < 0:
				lowerBound = 0
			if col > lowerBound:
				timePeriod = npa[row][lowerBound:col]
				high = timePeriod.max()
				low = timePeriod.min()
				mean = (high + low + timePeriod[-1]) / 3
				retrow.append([high, low, mean])
			col = lowerBound
		retrow.reverse()
		ret.append(retrow)
		row += 1
	return ret

#pps is single instrument pps list, overHowManyPPs is minimum pps to show the trend to qualify
#returs: price difference between the 2 end PPs closing means (same trend must be present overHowManyPPs or 0)
def trendOfPPs(pps, overHowManyPPs): 
	index = len(pps) - 1
	prevTrend = 0
	trend = 0

	#if parameters are out of shape return 0
	if index <= 0 or overHowManyPPs <= 1:
		return 0

	while index > 0 and overHowManyPPs > 1:
#		if pps[index][0] > pps[index - 1][0] and pps[index][1] > pps[index - 1][1] and pps[index][2] > pps[index - 1][2]:
#			trend = 1
#		elif pps[index][0] < pps[index - 1][0] and pps[index][1] < pps[index - 1][1] and pps[index][2] < pps[index - 1][2]:
#			trend = -1
		if pps[index][2] > pps[index - 1][2]:
			trend = 1
		elif pps[index][2] < pps[index - 1][2]:
			trend = -1
		else:
			trend = 0
		if trend != 0 and (prevTrend == 0 or trend == prevTrend):
			prevTrend = trend
			index -= 1
			overHowManyPPs -= 1
		else:
			return 0
	return pps[-1][2] - pps[index][2]

@backtest(200)
def getMyPosition(prcSoFar):
	global currentPos, firstCall, openPositions
	(nins, nt) = prcSoFar.shape
	if (nt < 2):
		return np.zeros(nins)
	lastRet = np.log(prcSoFar[:, -2] / prcSoFar[:, -1])
	lNorm = np.sqrt(lastRet.dot(lastRet))
	lastRet /= lNorm
	(means, stddev) = halfMeans(prcSoFar)
	PPs = calcPivotPoints(prcSoFar, 5)
#not used
	if firstCall:
		firstCall = False
		for i in range(nins):
			meansTrend = means[1][i] - means[0][i]
			openPositions[i].append([int(POSSTART1 * meansTrend / prcSoFar[i, -1]), prcSoFar[i, -1]])
#end not used
	rpos = []
#	print()
#	print(PPs)
	for i in range(nins):

		meansTrend = means[1][i] - means[0][i]
		priceDiff = prcSoFar[i, -1] - means[1][i]
		abs_priceDiff = abs(priceDiff)

		ppsTrend = trendOfPPs(PPs[i], 8)
		ppsTrendShort = trendOfPPs(PPs[i], 3)
		unitPPsTrend = ppsTrend
		unitPPsTrendShort = ppsTrendShort 
#		print(ppsTrend)
		if ppsTrend != 0:
			unitPPsTrend = ppsTrend / abs(ppsTrend)
		if ppsTrendShort != 0:
			unitPPsTrendShort = ppsTrendShort / abs(ppsTrendShort)
		#create new pos based on last half of the time average and stddev
		if True and stddev[1][i] > 0 and abs_priceDiff > 1 * stddev[1][i] and abs_priceDiff < 20 * stddev[1][i] and (True or ppsTrendShort > 0 and priceDiff < 0 or ppsTrendShort <= 0 and priceDiff > 0):# and len([1 for x in openPositions[i] if x[4] == 0]) == 0:
			pos = DESIREDRET / stddev[1][i] * (-priceDiff) / stddev[1][i]
#			pos = DESIREDRET / stddev[1][i] * (-priceDiff/abs_priceDiff) * 1
			if priceDiff > 0:
				lBound = means[1][i] + 0.0 * priceDiff
				uBound = 0
			else:
				lBound = 0
				uBound = means[1][i] + 0.0 * priceDiff
			openPositions[i].append([pos, prcSoFar[i, -1], lBound, uBound, 0])
		
		#create new pos based on PPs
#		print(PPs[i])
		if True and ppsTrend != 0 and len([1 for x in openPositions[i] if x[4] == 1]) == 0:
			openPositions[i].append([5000 / ppsTrend, prcSoFar[i, -1], 0, 0, 1])
#			print('OP', i, openPositions[i][-1], ppsTrend)
		# go through open positions per instrument adding them up per inst (and optionally removing / closing some)
		totalPosPerInst = 0
		newPosList = []
#		print(openPositions[i])
		for j in range(len(openPositions[i])):
			#keep pos trading
			pos = openPositions[i][j][0]
			openprice = openPositions[i][j][1]
			lb = openPositions[i][j][2]
			hb = openPositions[i][j][3]
			typ = openPositions[i][j][4]
			if typ == 0 and (prcSoFar[i, -1] > lb or lb == 0) and (prcSoFar[i, -1] < hb or hb == 0) or \
				typ == 1 and pos / abs(pos) == unitPPsTrendShort:
				#modify means reversal positions based on trend - risk mitigation
				if typ == 0:
					if ppsTrend > 0 and pos < 0:
						openPositions[i][j][2] += (openprice - lb) * .8
					if ppsTrend < 0 and pos > 0:
						openPositions[i][j][3] -= (hb - openprice) * .8
				newPosList.append(openPositions[i][j])
				totalPosPerInst += (openPositions[i][j][0])
#			else:
#				if typ == 1:
#					print('CL', i, openPositions[i][j], ppsTrendShort, 'PR', prcSoFar[i, -1])
		openPositions[i] = newPosList
#		print(totalPosPerInst, end=' ')
		#change total pos per instrument when it changes enough to avoid microtransactions
		dPos = totalPosPerInst - currentPos[i]
		if False or abs(dPos) > 5:
			rpos.append(int(totalPosPerInst))
		else:
			rpos.append(currentPos[i])
#		print('mean:', means[1][i], '\nprice:', prcSoFar[i, -1], '\nstddev:', stddev[1][i])
#		print('position list per inst:', openPositions[i])
#		print(format(len(openPositions[i]), '>3d'), end='')
		print(rpos[i], end=' ')
	print()
#		print('rpos:', rpos)

	currentPos = np.array(rpos)
#	rpos = np.array([int(x) for x in 100 * lastRet / prcSoFar[:, -1]])
#	currentPos = np.array([int(x) for x in currentPos+rpos])
	return currentPos


Testing function: getMyPosition
-9 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1251 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -5 
-21 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1251 0 -5 0 0 -9 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -10 
Day 549 value: -32.01 todayPL: $-26.75 $-traded: 11655 return: -0.00275
-31 0 0 0 0 6 7 -6 6 0 0 0 0 0 0 0 0 0 0 8 0 0 0 0 1251 0 -5 6 -6 -9 0 5 0 14 0 6 -5 0 0 -7 0 0 0 5 0 0 0 0 -6 -15 
Day 550 value: -14.03 todayPL: $17.98 $-traded: 15978 return: -0.00088
-40 0 0 0 0 6 7 -6 6 5 0 0 0 0 0 0 0 0 0 22 0 8 -5 0 0 6 -11 6 -6 -17 0 5 0 19 0 6 -5 0 0 -7 0 0 0 5 0 0 0 0 -6 -15 
Day 551 value: -34.67 todayPL: $-20.64 $-traded: 28605 return: -0.00121
3283 0 0 0 5 6 7 -6 6 5 0 -5 0 0 0 0 0 0 0 33 0 17 -5 0 0 6 -11 6 -6 -17 6 5 0 24 0 6 -5 0 0 -7 0 5 0 5 0 0 0 0 -6 -22 
Day 552 value: -25.68 todayPL: $8.99 $-traded: 41983 return: -0.00061
3283 0 0 6 5 11 13 -12 6 5 0 -5 0 0 0 0 0 0 0 44 0 25 -5 0 0 6 -16 12 -12 -25 6 11 0 29 0 12 -11 0 0 -12 0 5 0 5 0

In [30]:
import numpy as np

# Configuration constants
NUM_INSTRUMENTS = 50
DESIRED_RETURN_TARGET = 3
PIVOT_POINT_WINDOW = 5
MIN_POSITION_CHANGE_THRESHOLD = 5

# Global state variables
current_positions = np.zeros(NUM_INSTRUMENTS, int)
open_positions_per_stock = [[] for _ in range(NUM_INSTRUMENTS)]
# Each position: [position_size, entry_price, lower_bound, upper_bound, position_type]
# position_type: 0 = mean reversion, 1 = trend following
# bound == 0 means bound inactive

def calculate_half_period_statistics(price_array):
    """
    Split price history in half and calculate means/std for each half
    Returns: ([first_half_means, second_half_means], [first_half_stds, second_half_stds])
    """
    num_instruments, num_days = price_array.shape
    
    if num_days <= 1:
        return ([price_array, price_array], [np.zeros(num_instruments), np.zeros(num_instruments)])
    
    # Split data in half
    if num_days % 2 == 0:
        first_half_days = num_days // 2
        second_half_start = first_half_days
    else:
        first_half_days = num_days // 2 + 1
        second_half_start = first_half_days - 1
    
    means = []
    standard_deviations = []
    
    # Calculate statistics for each half
    means.append(price_array[:, :first_half_days].mean(axis=1))
    means.append(price_array[:, second_half_start:].mean(axis=1))
    standard_deviations.append(price_array[:, :first_half_days].std(axis=1, ddof=1))
    standard_deviations.append(price_array[:, second_half_start:].std(axis=1, ddof=1))
    
    return (means, standard_deviations)

def calculate_pivot_points(price_array, days_per_pivot):
    """
    Calculate pivot points for each stock over rolling windows
    Returns: List of pivot points per stock, each pivot: [high, low, pivot_mean]
    pivot_mean = (high + low + closing_price) / 3
    """
    num_instruments, num_days = price_array.shape
    
    if days_per_pivot < 2 or num_instruments == 0 or num_days == 0:
        return [[[price, price, price] for price in stock_prices] for stock_prices in price_array]
    
    all_pivot_points = []
    
    for stock_idx in range(num_instruments):
        stock_pivot_points = []
        current_day = num_days
        
        # Work backwards through time periods
        while current_day > 0:
            period_start = max(0, current_day - days_per_pivot)
            
            if current_day > period_start:
                period_prices = price_array[stock_idx][period_start:current_day]
                high_price = period_prices.max()
                low_price = period_prices.min()
                closing_price = period_prices[-1]
                pivot_mean = (high_price + low_price + closing_price) / 3
                
                stock_pivot_points.append([high_price, low_price, pivot_mean])
            
            current_day = period_start
        
        # Reverse to get chronological order
        stock_pivot_points.reverse()
        all_pivot_points.append(stock_pivot_points)
    
    return all_pivot_points

def calculate_pivot_trend_strength(pivot_points_list, min_consecutive_periods):
    """
    Calculate trend strength from pivot points
    Returns: Price difference between trend endpoints (0 if no consistent trend)
    """
    if len(pivot_points_list) <= 1 or min_consecutive_periods <= 1:
        return 0
    
    current_index = len(pivot_points_list) - 1
    previous_trend_direction = 0
    consecutive_periods_found = 0
    
    while current_index > 0 and consecutive_periods_found < min_consecutive_periods - 1:
        current_pivot_mean = pivot_points_list[current_index][2]
        previous_pivot_mean = pivot_points_list[current_index - 1][2]
        
        # Determine trend direction
        if current_pivot_mean > previous_pivot_mean:
            current_trend_direction = 1  # Uptrend
        elif current_pivot_mean < previous_pivot_mean:
            current_trend_direction = -1  # Downtrend
        else:
            current_trend_direction = 0  # No trend
        
        # Check if trend continues
        if current_trend_direction != 0 and (previous_trend_direction == 0 or current_trend_direction == previous_trend_direction):
            previous_trend_direction = current_trend_direction
            current_index -= 1
            consecutive_periods_found += 1
        else:
            return 0  # Trend broken
    
    # Return price difference if consistent trend found
    if consecutive_periods_found >= min_consecutive_periods - 1:
        return pivot_points_list[-1][2] - pivot_points_list[current_index][2]
    else:
        return 0

@backtest(200, stock_list=[0, 2, 3, 4, 5, 7, 8, 10, 11, 12, 16, 17, 18, 19, 20, 21, 22, 23, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39, 40, 41, 42, 43, 44, 46, 48, 49])
def getMyPosition(price_history_so_far):
    """
    Multi-strategy trading algorithm combining mean reversion and trend following
    """
    global current_positions, open_positions_per_stock
    
    num_instruments, num_days = price_history_so_far.shape
    if num_days < 2:
        return np.zeros(num_instruments)
    
    # Calculate technical indicators
    half_period_stats = calculate_half_period_statistics(price_history_so_far)
    recent_means, recent_std_devs = half_period_stats
    
    all_pivot_points = calculate_pivot_points(price_history_so_far, PIVOT_POINT_WINDOW)
    
    new_positions = []
    
    # Process each stock
    for stock_idx in range(num_instruments):
        current_price = price_history_so_far[stock_idx, -1]
        recent_mean = recent_means[1][stock_idx]  # Second half mean
        recent_volatility = recent_std_devs[1][stock_idx]  # Second half std dev
        
        price_deviation = current_price - recent_mean
        abs_price_deviation = abs(price_deviation)
        
        # Calculate trend indicators
        long_term_trend = calculate_pivot_trend_strength(all_pivot_points[stock_idx], 8)
        short_term_trend = calculate_pivot_trend_strength(all_pivot_points[stock_idx], 3)
        
        # Normalize trends to unit direction
        long_term_trend_direction = np.sign(long_term_trend) if long_term_trend != 0 else 0
        short_term_trend_direction = np.sign(short_term_trend) if short_term_trend != 0 else 0
        
        # STRATEGY 1: Mean Reversion (Type 0 positions)
        # Open position when price deviates significantly from recent mean
        deviation_threshold_min = 1 * recent_volatility
        deviation_threshold_max = 20 * recent_volatility
        
        if (recent_volatility > 0 and 
            deviation_threshold_min < abs_price_deviation < deviation_threshold_max):
            
            # Position size based on deviation and volatility
            position_size = DESIRED_RETURN_TARGET / recent_volatility * (-price_deviation) / recent_volatility
            
            # Set exit bounds
            if price_deviation > 0:  # Price above mean -> short position
                lower_bound = recent_mean  # Exit when price returns to mean
                upper_bound = 0  # No upper bound
            else:  # Price below mean -> long position
                lower_bound = 0  # No lower bound
                upper_bound = recent_mean  # Exit when price returns to mean
            
            # Add mean reversion position
            open_positions_per_stock[stock_idx].append([
                position_size, current_price, lower_bound, upper_bound, 0
            ])
        
        # STRATEGY 2: Trend Following (Type 1 positions)
        # Open position based on long-term pivot point trends
        existing_trend_positions = [pos for pos in open_positions_per_stock[stock_idx] if pos[4] == 1]
        
        if long_term_trend != 0 and len(existing_trend_positions) == 0:
            # Position size inversely proportional to trend strength (risk management)
            trend_position_size = 5000 / long_term_trend
            
            # Add trend following position (no bounds, exit on trend reversal)
            open_positions_per_stock[stock_idx].append([
                trend_position_size, current_price, 0, 0, 1
            ])
        
        # POSITION MANAGEMENT: Review and close positions
        total_position_for_stock = 0
        remaining_positions = []
        
        for position in open_positions_per_stock[stock_idx]:
            position_size, entry_price, lower_bound, upper_bound, position_type = position
            
            # Determine if position should remain open
            keep_position = False
            
            if position_type == 0:  # Mean reversion position
                # Keep if within bounds
                within_lower_bound = (current_price > lower_bound) or (lower_bound == 0)
                within_upper_bound = (current_price < upper_bound) or (upper_bound == 0)
                keep_position = within_lower_bound and within_upper_bound
                
                # Adjust bounds based on long-term trend (risk management)
                if keep_position:
                    if long_term_trend > 0 and position_size < 0:  # Long trend, short position
                        # Tighten lower bound to reduce risk
                        position[2] += (entry_price - lower_bound) * 0.8
                    elif long_term_trend < 0 and position_size > 0:  # Short trend, long position
                        # Tighten upper bound to reduce risk
                        position[3] -= (upper_bound - entry_price) * 0.8
            
            elif position_type == 1:  # Trend following position
                # Keep if short-term trend matches position direction
                position_direction = np.sign(position_size)
                keep_position = (position_direction == short_term_trend_direction)
            
            if keep_position:
                remaining_positions.append(position)
                total_position_for_stock += position_size
        
        # Update open positions
        open_positions_per_stock[stock_idx] = remaining_positions
        
        # Apply transaction cost threshold
        position_change = total_position_for_stock - current_positions[stock_idx]
        
        if abs(position_change) > MIN_POSITION_CHANGE_THRESHOLD:
            new_positions.append(int(total_position_for_stock))
        else:
            new_positions.append(current_positions[stock_idx])
        
        # Debug output (optional)
        print(new_positions[stock_idx], end=' ')
    
    print()  # New line after all positions printed
    
    # Update global state
    current_positions = np.array(new_positions)
    return current_positions

Testing function: getMyPosition
Evaluating subset: 38 stocks [0, 2, 3, 4, 5, 7, 8, 10, 11, 12, 16, 17, 18, 19, 20, 21, 22, 23, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39, 40, 41, 42, 43, 44, 46, 48, 49]
-9 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1251 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -5 
-21 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1251 0 -5 0 0 -9 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 -10 
Day 549 value: -32.01 todayPL: $-26.75 $-traded: 11655 return: -0.00275
-31 0 0 0 0 6 7 -6 6 0 0 0 0 0 0 0 0 0 0 8 0 0 0 0 1251 0 -5 6 -6 -9 0 5 0 14 0 6 -5 0 0 -7 0 0 0 5 0 0 0 0 -6 -15 
Day 550 value: -14.03 todayPL: $17.98 $-traded: 15978 return: -0.00088
-40 0 0 0 0 6 7 -6 6 5 0 0 0 0 0 0 0 0 0 22 0 8 -5 0 0 6 -11 6 -6 -17 0 5 0 19 0 6 -5 0 0 -7 0 0 0 5 0 0 0 0 -6 -15 
Day 551 value: -34.67 todayPL: $-20.64 $-traded: 28605 return: -0.00121
3283 0 0 0 5 6 7 -6 6 5 0 -5 0 0 0 0 0 0 0 33 0 17 -5 0 0 6 -11 6 -6 -17 6 5 0 24 0 6 -5 0 0 -7 0 5 0 5 0 0 0 0 -6 -22 
Day 552 v

In [32]:
@backtest(200, stock_list=[0, 2, 3, 4, 5, 7, 8, 10, 11, 12, 16, 17, 18, 19, 20, 21, 22, 23, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39, 40, 41, 42, 43, 44, 46, 48, 49])
def getMyPosition(price_history_so_far):
    """
    Multi-strategy trading algorithm combining mean reversion and trend following
    """
    global current_positions, open_positions_per_stock
    
    num_instruments, num_days = price_history_so_far.shape
    if num_days < 2:
        return np.zeros(num_instruments)
    
    # Define which stocks to actually trade (must match stock_list above!)
    tradeable_stocks = [0, 2, 3, 4, 5, 7, 8, 10, 11, 12, 16, 17, 18, 19, 20, 21, 22, 23, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39, 40, 41, 42, 43, 44, 46, 48, 49]
    
    # Calculate technical indicators
    half_period_stats = calculate_half_period_statistics(price_history_so_far)
    recent_means, recent_std_devs = half_period_stats
    
    all_pivot_points = calculate_pivot_points(price_history_so_far, PIVOT_POINT_WINDOW)
    
    positions = np.zeros(num_instruments)  # Initialize all positions to 0
    
    # Process ONLY the stocks we want to trade
    for stock_idx in tradeable_stocks:  # ← Fixed: only iterate over tradeable stocks
        current_price = price_history_so_far[stock_idx, -1]
        recent_mean = recent_means[1][stock_idx]  # Second half mean
        recent_volatility = recent_std_devs[1][stock_idx]  # Second half std dev
        
        price_deviation = current_price - recent_mean
        abs_price_deviation = abs(price_deviation)
        
        # Calculate trend indicators
        long_term_trend = calculate_pivot_trend_strength(all_pivot_points[stock_idx], 8)
        short_term_trend = calculate_pivot_trend_strength(all_pivot_points[stock_idx], 3)
        
        # Normalize trends to unit direction
        long_term_trend_direction = np.sign(long_term_trend) if long_term_trend != 0 else 0
        short_term_trend_direction = np.sign(short_term_trend) if short_term_trend != 0 else 0
        
        # STRATEGY 1: Mean Reversion (Type 0 positions)
        deviation_threshold_min = 1 * recent_volatility
        deviation_threshold_max = 20 * recent_volatility
        
        if (recent_volatility > 0 and 
            deviation_threshold_min < abs_price_deviation < deviation_threshold_max):
            
            position_size = DESIRED_RETURN_TARGET / recent_volatility * (-price_deviation) / recent_volatility
            
            if price_deviation > 0:  # Price above mean -> short position
                lower_bound = recent_mean
                upper_bound = 0
            else:  # Price below mean -> long position
                lower_bound = 0
                upper_bound = recent_mean
            
            open_positions_per_stock[stock_idx].append([
                position_size, current_price, lower_bound, upper_bound, 0
            ])
        
        # STRATEGY 2: Trend Following (Type 1 positions)
        existing_trend_positions = [pos for pos in open_positions_per_stock[stock_idx] if pos[4] == 1]
        
        if long_term_trend != 0 and len(existing_trend_positions) == 0:
            trend_position_size = 5000 / long_term_trend
            open_positions_per_stock[stock_idx].append([
                trend_position_size, current_price, 0, 0, 1
            ])
        
        # POSITION MANAGEMENT: Review and close positions
        total_position_for_stock = 0
        remaining_positions = []
        
        for position in open_positions_per_stock[stock_idx]:
            position_size, entry_price, lower_bound, upper_bound, position_type = position
            keep_position = False
            
            if position_type == 0:  # Mean reversion position
                within_lower_bound = (current_price > lower_bound) or (lower_bound == 0)
                within_upper_bound = (current_price < upper_bound) or (upper_bound == 0)
                keep_position = within_lower_bound and within_upper_bound
                
                if keep_position:
                    if long_term_trend > 0 and position_size < 0:
                        position[2] += (entry_price - lower_bound) * 0.8
                    elif long_term_trend < 0 and position_size > 0:
                        position[3] -= (upper_bound - entry_price) * 0.8
            
            elif position_type == 1:  # Trend following position
                position_direction = np.sign(position_size)
                keep_position = (position_direction == short_term_trend_direction)
            
            if keep_position:
                remaining_positions.append(position)
                total_position_for_stock += position_size
        
        open_positions_per_stock[stock_idx] = remaining_positions
        
        # Apply transaction cost threshold
        position_change = total_position_for_stock - current_positions[stock_idx]
        
        if abs(position_change) > MIN_POSITION_CHANGE_THRESHOLD:
            positions[stock_idx] = int(total_position_for_stock)
        else:
            positions[stock_idx] = current_positions[stock_idx]
    
    # Update global state (only for tradeable stocks)
    for stock_idx in tradeable_stocks:
        current_positions[stock_idx] = positions[stock_idx]
    
    return positions

Testing function: getMyPosition
Evaluating subset: 38 stocks [0, 2, 3, 4, 5, 7, 8, 10, 11, 12, 16, 17, 18, 19, 20, 21, 22, 23, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 38, 39, 40, 41, 42, 43, 44, 46, 48, 49]
Day 549 value: -164.93 todayPL: $-118.41 $-traded: 96614 return: -0.00171
Day 550 value: -330.38 todayPL: $-165.46 $-traded: 100244 return: -0.00330
Day 551 value: -122.19 todayPL: $208.19 $-traded: 104785 return: -0.00117
Day 552 value: -144.67 todayPL: $-22.47 $-traded: 122050 return: -0.00119
Day 553 value: -670.95 todayPL: $-526.28 $-traded: 126073 return: -0.00532
Day 554 value: -348.72 todayPL: $322.22 $-traded: 163305 return: -0.00214
Day 555 value: -297.07 todayPL: $51.65 $-traded: 178163 return: -0.00167
Day 556 value: -1260.56 todayPL: $-963.49 $-traded: 181594 return: -0.00694
Day 557 value: -646.24 todayPL: $614.31 $-traded: 204127 return: -0.00317
Day 558 value: -904.59 todayPL: $-258.35 $-traded: 223089 return: -0.00405
Day 559 value: -625.58 todayPL: $279.02 $-traded:

In [ ]:
# Strategy: Segregating the stocks into categories:
# Volatility: Low risk, mid risk, high risk
# High correlation, positive/negative correlation

# Extends the eval function to give out the sharpe ratio
# Based on the visualization, we have:

# 2 and 6: positive correlation, 6 lags behind by about 10 days
# 4 and 20: negative correlation, 20 lags behind by ahout 10 days
# 4 and 6: strong positive correlation, 4 lags behind 6 by about 20 days
# 11 and 23: positive correlation
# 29 and 42: strong positive correlation
# 4 and 15: strong negative correlation
# 16 and 20: strong positive correlation
# 6 and 20: strong positive correlation, 20 lags behind by 30 days
# 2 and 16: strong positive correlation
# 20 and 47: positive correlation

# Ranges:

In [3]:
data = pd.read_csv(pricesFile, header=None, sep=r"\s+", names=[i for i in range(50)])
data

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,37.64,69.39,63.26,31.56,61.85,28.67,37.09,43.23,41.24,70.36,...,75.24,41.38,30.08,38.24,75.35,28.59,73.75,23.97,75.71,27.74
1,37.70,69.71,64.04,31.42,62.02,28.42,37.09,42.90,41.40,69.12,...,75.33,41.69,30.34,37.85,75.21,28.55,74.32,24.01,75.77,27.54
2,37.67,69.90,64.37,31.01,62.22,28.66,36.29,42.64,41.18,68.31,...,75.38,41.34,30.68,37.54,75.18,28.38,75.17,24.32,75.58,27.54
3,37.60,69.62,65.54,31.72,62.07,28.47,36.55,42.45,42.32,67.00,...,74.95,41.33,30.65,37.18,75.42,28.28,75.69,24.26,76.22,27.69
4,37.60,69.57,64.89,31.75,61.81,28.29,35.79,42.29,42.52,66.53,...,75.69,41.08,30.32,37.48,75.39,28.43,76.42,24.36,75.42,27.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,36.49,51.34,45.11,29.37,39.73,28.69,17.89,47.71,40.70,58.56,...,75.13,46.74,39.54,32.21,75.55,30.62,85.74,16.13,81.57,31.53
743,36.30,50.95,44.66,29.32,39.85,28.59,17.84,47.31,40.32,58.05,...,75.06,47.19,39.32,32.92,75.97,30.87,85.88,15.91,80.50,31.65
744,36.58,50.80,44.25,29.17,40.01,28.97,17.76,47.01,39.97,57.61,...,74.97,47.52,39.58,33.31,76.05,30.83,86.15,15.81,80.93,31.83
745,36.43,51.19,44.02,29.11,40.01,28.54,17.69,45.68,39.86,59.06,...,75.36,48.15,39.94,32.95,75.38,30.81,86.42,15.75,80.80,31.78


In [2]:
# Sort by variance
high_var = [23, 9, 22, 2, 14, 4, 11, 16, 6, 28, 46, 7, 29, 31, 42, 30, 1]
med_var = [32, 41, 12, 25, 20, 37, 21, 39, 35, 44, 27, 34, 43, 24, 36, 13, 40, 47, 17, 18, 10, 26, 8, 3, 38]
low_var = [33, 48, 15, 49, 19, 5, 0, 45]

low_var_good = [33, 48, 15, 19, 0, 45]

In [15]:
# Global tracking variables
previous_positions = np.zeros(len(low_var_good))
trade_day_counter = 0
is_first_day = True

@backtest(numTestDays=747,stock_list=low_var_good, show_daily_positions=True)
def low_var_strat(prcHistSoFar):
    """
    Plan:
    - make a simple linear regression of the last 100 days
    - if first day:
        - if positive slope, buy to target_dollar_exposure
        - if negative slope, sell to target_dollar_exposure
    - else:
        - if slope continues to be positive, buy by 50*slope
        - if slope is negative, sell 30*slope
    make trade every 20 days
    """
    global previous_positions, trade_day_counter, is_first_day
    
    nInst, nt = prcHistSoFar.shape
    
    if nt < 100:  # Need sufficient history for 100-day regression
        return np.zeros(nInst)
    
    from scipy import stats
    
    target_dollar_exposure = 2000  # Target dollar amount per stock
    linreg_window = 20  # Use 100 days or all available data
    
    positions = np.zeros(nInst)
    
    # Only trade every 20 days (except first day)
    trade_day_counter += 1
    if not is_first_day and trade_day_counter % 3 != 0:
        return previous_positions.copy()
    
    # Process each instrument in low_var list
    for i, stock_idx in enumerate(low_var):
        if stock_idx >= nInst:  # Safety check
            continue
            
        prices = prcHistSoFar[stock_idx, :]
        current_price = prices[-1]
        
        # Linear regression on last 100 days (or available data)
        x_vals = np.arange(linreg_window)
        y_vals = prices[-linreg_window:]
        
        slope = stats.linregress(x_vals, y_vals)[0]
        
        # Normalize slope by current price to get daily % change
        normalized_slope = slope / current_price
        
        if is_first_day:
            # FIRST DAY LOGIC
            if normalized_slope > 0:
                # Positive slope: buy to target dollar exposure
                target_shares = target_dollar_exposure / current_price
                positions[stock_idx] = int(target_shares)
            elif normalized_slope < 0:
                # Negative slope: sell to target dollar exposure  
                target_shares = target_dollar_exposure / current_price
                positions[stock_idx] = int(-target_shares)
            else:
                # Flat trend: no position
                positions[stock_idx] = 0
        else:
            # SUBSEQUENT DAYS LOGIC
            previous_pos = previous_positions[stock_idx]
            
            if normalized_slope > 0:
                # Positive slope continues: buy by 50*slope
                additional_shares = int(50 * normalized_slope * current_price / current_price)
                # Ensure we don't exceed reasonable position limits
                max_additional = target_dollar_exposure // current_price
                additional_shares = min(additional_shares, max_additional)
                positions[stock_idx] = previous_pos + additional_shares
                
            elif normalized_slope < 0:
                # Negative slope: sell by 30*slope  
                additional_shares = int(32 * abs(normalized_slope) * current_price / current_price)
                max_additional = target_dollar_exposure // current_price
                additional_shares = min(additional_shares, max_additional)
                positions[stock_idx] = previous_pos - additional_shares
                
            else:
                # No clear trend: maintain position
                positions[stock_idx] = previous_pos
        
        # Apply position limits based on dollar limit
        max_position = int(dlrPosLimit / current_price)
        positions[stock_idx] = np.clip(positions[stock_idx], -max_position, max_position)
    
    # Update global state
    previous_positions = positions.copy()
    is_first_day = False
    
    return positions

Testing function: low_var_strat
Evaluating subset: 6 stocks [33, 48, 15, 19, 0, 45]
Day 2 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
  Cash: $0.00 | Position Value: $0.00
  Active Positions: None

Day 3 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
  Cash: $0.00 | Position Value: $0.00
  Active Positions: None

Day 4 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
  Cash: $0.00 | Position Value: $0.00
  Active Positions: None

Day 5 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
  Cash: $0.00 | Position Value: $0.00
  Active Positions: None

Day 6 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
  Cash: $0.00 | Position Value: $0.00
  Active Positions: None

Day 7 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
  Cash: $0.00 | Position Value: $0.00
  Active Positions: None

Day 8 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
  Cash: $0.00 | Position Value: $0.00
  Active Positions: None

Day 9 value: 0.00 todayPL: $0.00 $-traded

In [24]:
# Global tracking variables
previous_positions = np.zeros(len(low_var_good))
trade_day_counter = 0
is_first_day = True

@backtest(numTestDays=200, stock_list=low_var_good, show_daily_positions=True)
def low_var_strat(prcHistSoFar):
    """
    Plan:
    - make a simple linear regression of the last 20 days
    - ADD: mean reversion component to adjust position sizes
    - if first day:
        - if positive slope, buy to target_dollar_exposure (adjusted by mean reversion)
        - if negative slope, sell to target_dollar_exposure (adjusted by mean reversion)
    - else:
        - if slope continues to be positive, buy by 50*slope (adjusted by mean reversion)
        - if slope is negative, sell 30*slope (adjusted by mean reversion)
    make trade every 3 days
    """
    global previous_positions, trade_day_counter, is_first_day
    
    nInst, nt = prcHistSoFar.shape
    
    if nt < 100:  # Need sufficient history
        return np.zeros(nInst)
    
    from scipy import stats
    
    target_dollar_exposure = 100  # Base target dollar amount per stock
    linreg_window = 20  # Linear regression window
    mean_reversion_window = 30  # Mean reversion lookback window
    
    positions = np.zeros(nInst)
    
    # Only trade every 3 days (except first day)
    trade_day_counter += 1
    if not is_first_day and trade_day_counter % 3 != 0:
        return previous_positions.copy()
    
    # Process each instrument in low_var list
    for i, stock_idx in enumerate(low_var):
        if stock_idx >= nInst:  # Safety check
            continue
            
        prices = prcHistSoFar[stock_idx, :]
        current_price = prices[-1]
        
        # LINEAR REGRESSION COMPONENT
        x_vals = np.arange(linreg_window)
        y_vals = prices[-linreg_window:]
        slope = stats.linregress(x_vals, y_vals)[0]
        normalized_slope = slope / current_price
        
        # MEAN REVERSION COMPONENT
        # Calculate mean and standard deviation over longer window
        mean_window = min(mean_reversion_window, nt)
        recent_mean = np.mean(prices[-mean_window:])
        recent_std = np.std(prices[-mean_window:])
        
        # Calculate how far current price deviates from mean (in standard deviations)
        if recent_std > 0:
            z_score = (current_price - recent_mean) / recent_std
        else:
            z_score = 0
        
        # Mean reversion multiplier
        # If price is above mean (z_score > 0), reduce long positions, increase short positions
        # If price is below mean (z_score < 0), increase long positions, reduce short positions
        mean_reversion_multiplier = 1.0 - (z_score * 0.2)  # Adjust by up to ±20%
        mean_reversion_multiplier = np.clip(mean_reversion_multiplier, 0.3, 1.7)  # Keep reasonable bounds
        
        # Adjust target exposure based on mean reversion
        adjusted_target_exposure = target_dollar_exposure * mean_reversion_multiplier
        
        if is_first_day:
            # FIRST DAY LOGIC with mean reversion adjustment
            if normalized_slope > 0:
                # Positive slope: buy to adjusted target dollar exposure
                target_shares = adjusted_target_exposure / current_price
                positions[stock_idx] = int(target_shares)
            elif normalized_slope < 0:
                # Negative slope: sell to adjusted target dollar exposure  
                target_shares = adjusted_target_exposure / current_price
                positions[stock_idx] = int(-target_shares)
            else:
                # Flat trend: mean reversion only
                if z_score > 1.0:  # Price too high, short
                    target_shares = (adjusted_target_exposure * 0.5) / current_price
                    positions[stock_idx] = int(-target_shares)
                elif z_score < -1.0:  # Price too low, long
                    target_shares = (adjusted_target_exposure * 0.5) / current_price
                    positions[stock_idx] = int(target_shares)
                else:
                    positions[stock_idx] = 0
        else:
            # SUBSEQUENT DAYS LOGIC with mean reversion adjustment
            previous_pos = previous_positions[stock_idx]
            
            if normalized_slope > 0:
                # Positive slope continues: buy by 50*slope, adjusted by mean reversion
                base_additional = int(50 * normalized_slope * current_price / current_price)
                
                # Apply mean reversion: if price is too high, reduce buying
                if z_score > 0.5:  # Price above mean, reduce buying
                    additional_shares = int(base_additional * (1.0 - z_score * 0.3))
                else:  # Price at/below mean, normal or increased buying
                    additional_shares = int(base_additional * mean_reversion_multiplier)
                
                max_additional = int(adjusted_target_exposure // current_price)
                additional_shares = min(abs(additional_shares), max_additional)
                positions[stock_idx] = previous_pos + additional_shares
                
            elif normalized_slope < 0:
                # Negative slope: sell by 30*slope, adjusted by mean reversion
                base_additional = int(32 * abs(normalized_slope) * current_price / current_price)
                
                # Apply mean reversion: if price is too low, reduce selling
                if z_score < -0.5:  # Price below mean, reduce selling
                    additional_shares = int(base_additional * (1.0 + z_score * 0.3))
                else:  # Price at/above mean, normal or increased selling
                    additional_shares = int(base_additional * mean_reversion_multiplier)
                
                max_additional = int(adjusted_target_exposure // current_price)
                additional_shares = min(abs(additional_shares), max_additional)
                positions[stock_idx] = previous_pos - additional_shares
                
            else:
                # No clear trend: pure mean reversion strategy
                if abs(z_score) > 1.0:  # Strong deviation from mean
                    reversion_size = int((adjusted_target_exposure * 0.3) / current_price)
                    if z_score > 1.0:  # Price too high, add short position
                        positions[stock_idx] = previous_pos - reversion_size
                    else:  # Price too low, add long position
                        positions[stock_idx] = previous_pos + reversion_size
                else:
                    # Small deviation, maintain position
                    positions[stock_idx] = previous_pos
        
        # Apply position limits based on dollar limit
        max_position = int(dlrPosLimit / current_price)
        positions[stock_idx] = np.clip(positions[stock_idx], -max_position, max_position)
    
    # Update global state
    previous_positions = positions.copy()
    is_first_day = False
    
    return positions

Testing function: low_var_strat
Evaluating subset: 6 stocks [33, 48, 15, 19, 0, 45]
Day 549 value: 4.04 todayPL: $4.32 $-traded: 563 return: 0.00717
  Cash: $-55.51 | Position Value: $59.55
  Active Positions: S33:8@10.96=$88 | S15:-1@53.84=$-54 | S19:-3@29.58=$-89 | S0:2@36.52=$73 | S45:3@28.83=$86

Day 550 value: 3.39 todayPL: $-0.65 $-traded: 563 return: 0.00602
  Cash: $-55.51 | Position Value: $58.90
  Active Positions: S33:8@10.77=$86 | S15:-1@53.17=$-53 | S19:-3@29.23=$-88 | S0:2@36.26=$73 | S45:3@28.80=$86

Day 551 value: 3.15 todayPL: $-0.24 $-traded: 563 return: 0.00559
  Cash: $-55.51 | Position Value: $58.66
  Active Positions: S33:8@10.87=$87 | S15:-1@53.33=$-53 | S19:-3@28.92=$-87 | S0:2@36.21=$72 | S45:3@28.39=$85

Day 552 value: 1.10 todayPL: $-2.05 $-traded: 563 return: 0.00195
  Cash: $-55.51 | Position Value: $56.61
  Active Positions: S33:8@10.85=$87 | S15:-1@53.63=$-54 | S19:-3@29.02=$-87 | S0:2@36.10=$72 | S45:3@28.28=$85

Day 553 value: -0.48 todayPL: $-1.58 $-tr